### TODO: IMPOVE THIS

# KNN features vs errors
THE IDEA IS SEE IF THE VALUES OF SOME FEATURES HAS HIGHER ERRORS - training a knn model to identify which segments of data has the higger errors


Plots of predictions of the models with any kind of transformation

**IMPORTANT**: The list of models to evaluate is the same, but each model could have its own feature eng, but the Input (the data_X) and the Output (the prediction) follow the same structure, so it is necesary only one notebook to evaluate the differents notebooks of training (if it is not logic for you thinking in the kaggle competitions).

In this notebook, there are a parameter "folder_models" and in this folder are located the pkl of each model

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

## RUN

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

### 0. Global params

In [ ]:
# define folder where the models were saved. There are the same models accepted by gurobi but the feature eng changed

# list of folder with models = ['basic', 'scaler', 'poly_2', 'poly_3']
folder_models = 'basic'

### 1. Load data

In [ ]:
### DEFINE LIST FEARTURES - TARGET (order data to have the same order in the features always)
list_features = ['AveOccup', 'Latitude', 'Population', 'AveBedrms', 'HouseAge', 'Longitude', 'AveRooms', 'MedInc']
target = 'Price'

In [ ]:
### LOAD DATA
X_train = pd.read_pickle('artifacts/data/X_train.pkl')
X_test = pd.read_pickle('artifacts/data/X_test.pkl')
y_train = pd.read_pickle('artifacts/data/y_train.pkl')
y_test = pd.read_pickle('artifacts/data/y_test.pkl')

In [ ]:
print('shape data')
print('\n\n TRAIN')
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

print('\n\n TEST')
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

In [ ]:
# # sample - run fast
# X_train = X_train[0:100]
# y_train = y_train[0:100]

### 2. Load Models
Load all the models in a dictory

In [ ]:
## define list of models - list to have always the same order.
#### In this example, the strings in the list are the same with the models were saved
list_models_names = [
    "lr",
    "ridge",
    "lasso",
    
    "tree_simple",
    "tree_default",
    
    "rf_simple",
    "rf_default",

    "gb_simple",
    "gb_default",

    "xgb_simple",
    "xgb_default",

    "mlp_simple",
    "mlp_default"
]

In [ ]:
# define path to folder models
path_folder_models = f'artifacts/models/{folder_models}/'

In [ ]:
### load models
dict_models = {}
for model_name in list_models_names:
    print(f'loading model: {model_name}')
    path_model = path_folder_models + f'{model_name}.pkl'
    with open(path_model, 'rb') as artifact:
        dict_models[model_name] = pickle.load(artifact)

### 3. Train KNN
It is necessary train and test datasets

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

### 3.0 Define model and data set

In [ ]:
# select model
model = dict_models['lr']

# plot errors or abs(errors)
abs_error = True

#### 3.1 Generate errors

In [ ]:
# predict y_train_pred
y_train_pred = model.predict(X_train)
y_train_pred = pd.DataFrame(y_train_pred, index = y_train.index, columns = y_train.columns)

# predict y_test_pred
y_test_pred = model.predict(X_test)
y_test_pred = pd.DataFrame(y_test_pred, index = y_test.index, columns = y_test.columns)

In [ ]:
# calculate error train
errors_train = y_train - y_train_pred # error
errors_train_abs = np.abs(errors_train) # errors in abs value
if abs_error:
    errors_train_to_plot = errors_train_abs
else:
    errors_train_to_plot = errors_train


# calculate error test
errors_test = y_test - y_test_pred # error
errors_test_abs = np.abs(errors_test) # errors in abs value
if abs_error:
    errors_test_to_plot = errors_test_abs
else:
    errors_test_to_plot = errors_test

#### 3.2 scale data - knn sensibility to dimensions of data - euclidian distance

In [ ]:
# fit
scaler = MinMaxScaler()
scaler.fit(X_train)

# transform
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### 3.3 split errors

In [ ]:
### define bins to split data

# deciles = [i / 10 for i in range(1, 10)]
# bins_split_errors = errors_train_to_plot.quantile(deciles)
# bins_split_errors

In [ ]:
# split train
errors_train_decil = pd.qcut(errors_train_to_plot[target].values, q=10, labels=False) + 1
errors_train_decil

In [ ]:
# split test - to do using bins get in train

#### 3.4 train knn regressor

In [ ]:
# for index_neighbors in range(1, 30):
#     print('\n ------- neighbors: ', index_neighbors)


#     # define model
#     knn = KNeighborsRegressor(n_neighbors = index_neighbors, leaf_size = 30)

#     # train
#     knn.fit(X_train_scaled, errors_train_to_plot)

#     # predict
#     prediction_train = knn.predict(X_train_scaled)
#     #prediction_test = knn.predict(X_test_scaled)

#     # metrics
#     r2 = r2_score(errors_train_to_plot, prediction_train)
#     mse = mean_squared_error(errors_train_to_plot, prediction_train, squared = True)
#     rmse = mean_squared_error(errors_train_to_plot, prediction_train, squared = False)

#     print('r2: ', r2)
#     print('mse: ', mse)
#     print('rmse: ', rmse)

### 3.5 train knn classifier

In [ ]:
for index_neighbors in range(1, 30):
    print('\n ------- neighbors: ', index_neighbors)


    # define model
    knn = KNeighborsClassifier(n_neighbors = index_neighbors)

    # train
    knn.fit(X_train_scaled, errors_train_decil)

    # score
    acc = knn.score(X_train_scaled, errors_train_decil)
    print('acc: ', acc)